<a href="https://colab.research.google.com/github/amfakh/eapanicobacoba/blob/master/Scrape_Genius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sauce:

https://medium.com/analytics-vidhya/how-to-scrape-song-lyrics-a-gentle-python-tutorial-5b1d4ab351d2

https://www.promptcloud.com/blog/scraping-song-lyrics-using-python-from-genius/

In [ ]:
import urllib.request
import urllib.parse
import urllib.error
import ssl
import os
import re
import requests

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [ ]:
GENIUS_API_TOKEN='PXpvxuPl_XyMM02ly-gyw8bd7tyP1avUQLZVsR4ooRmhfbTDOWG8mEHez0DGygjp'

In [ ]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response# Get Genius.com song url's from artist object
    
# DEMO
# request_song_url('JKT48', 10)

In [ ]:
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()        # Collect up to song_cap song objects from artist
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Collect song URL's from song objects
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)
            
        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Preparing to scrape {} songs by {}'.format(len(songs), artist_name))
    return songs

In [ ]:
# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [ ]:
song_json = {}
song_json["Primary Artist"] = []
song_json["Song Title"] = []
song_json["Lyrics"] = []


def scrape_from_link(url):

    req = Request(url, headers = { 'User-Agent' : 'Mozilla/5.0' })
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')

    for div in soup.findAll('h1', attrs = {'class': 'header_with_cover_art-primary_info-title'}):
        song_json["Song Title"].append(div.text.strip());
        print("    - Scrapped: " + song_json["Song Title"][-1])

    for div in soup.findAll('a', attrs = {'class': 'header_with_cover_art-primary_info-primary_artist'}):
        song_json["Primary Artist"].append(div.text.strip());    

    for div in soup.findAll('div', attrs = {'class': 'lyrics'}):
        song_json["Lyrics"].append(div.text.strip().split("\n"));

    # print(song_json["Song Title"])

In [ ]:
# scrape_from_link('https://genius.com/Jkt48-only-today-lyrics')

In [ ]:
# song_json

In [ ]:
import json

def write_lyrics_to_file(artist_name, song_count):
    # song_json["Primary Artist"].clear()
    # song_json["Song Title"].clear()
    # song_json["Lyrics"].clear()

    with open(artist_name.lower() + '.json', 'w') as outfile:
        urls = request_song_url(artist_name, song_count)
        for url in urls:
            # print(url)
            scrape_from_link(url)
      
        json.dump(song_json, outfile, indent = 4, ensure_ascii = False)
        print("\nFinished Succesfully")
  
# DEMO  
write_lyrics_to_file('jkt48', 100)

Preparing to scrape 100 songs by jkt48
    - Scrapped: Heavy Rotation
    - Scrapped: Fortune Cookie Yang Mencinta (Fortune Cookie In Love)
    - Scrapped: RIVER
    - Scrapped: Pesawat Kertas 365 Hari
    - Scrapped: Everyday, Kachuusha
    - Scrapped: Dirimu Melody (Kimi Wa Melody)
    - Scrapped: Only Today
    - Scrapped: Baby! Baby! Baby!
    - Scrapped: First Rabbit
    - Scrapped: Sebagian Besar Kenangan (Omoide No Hotondo)
    - Scrapped: Rapsodi
    - Scrapped: Seesaw Game Penuh Air Mata (Namida no Seesaw Game)
    - Scrapped: Kereta Kedewasaan (Otona Ressha)
    - Scrapped: So Long!
    - Scrapped: Bersama Kamu, Pelangi Dan Mentari (Kimi To Niji To Taiyou To)
    - Scrapped: Beginner
    - Scrapped: High Tension
    - Scrapped: Kita Pernah Di Sini (Koko Ni Itakoto)
    - Scrapped: Refrain Penuh Harapan (Kibouteki Refrain)
    - Scrapped: UZA
    - Scrapped: Sweet & Bitter
    - Scrapped: BINGO!
    - Scrapped: Di Bawah Langit Berwarna Sakura
    - Scrapped: Siapa yang Mempert

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json(r'jkt48.json')

In [ ]:
df

,Primary Artist,Song Title,Lyrics
0,JKT48,Heavy Rotation,"[[Intro], One! Two! Three! Four, , [Chorus 1],..."
1,JKT48,Fortune Cookie Yang Mencinta (Fortune Cookie I...,"[[Verse 1], Walaupun diri ini menyukaimu, Kamu..."
2,JKT48,RIVER,"[[Intro], JKT... 48!, , [Refrain], Majulah ke ..."
3,JKT48,Pesawat Kertas 365 Hari,"[[Verse 1], Kutatap langit di pagi hari, Kuawa..."
4,JKT48,"Everyday, Kachuusha","[[Verse 1], Saat mentari mulai bersinar, Lebih..."
...,...,...,...
95,JKT48,Mammoth,"[[Intro], We're mammoth!, We’re mammoth!, , [V..."
96,JKT48,Bel Terakhir Berbunyi (Saishuu Bell Ga Naru),"[[Verse 1], Larut malam peron stasiun kereta, ..."
97,JKT48,Dewi Theater (Theater No Megami),"[[Verse 1], Aku dengar dari orang-orang, Bahwa..."
98,JKT48,Natasha yang Ku Cinta (Itoshiki Natasha),"[[Verse 1], Ada seorang pria diam-diam menyeli..."
